In [1]:
#制作数据集tf.data.Dataset(可迭代element)
#tf.data.Dataset.from_tensor_slices() 数据量比较小能够装进内存
#! pip install tensorflow==2.0.0
#! pip uninstall tensorflow==1.2.1
import tensorflow as tf
import numpy as np
print(tf.__version__)
x=tf.constant([1,2,3,4,5])
y=tf.constant([6,7,8,9,10])
dataset=tf.data.Dataset.from_tensor_slices((x,y))
for x,y in dataset:
    print(x.numpy(),y.numpy())

2.0.0
1 6
2 7
3 8
4 9
5 10


In [2]:
def read_image_filenames(data_dir):
    cat_dir=data_dir+'cat/'
    dog_dir=data_dir+'dog/'
    cat_filenames=tf.constant([cat_dir+fn for fn in os.listdir(cat_dir)])
    dog_filenames=tf.constant([dog_dir+ fn for fn in os.listdir(dog_dir)])
    filenames=tf.concat([cat_filenames,dog_filenames],axis=-1)
    labels=tf.concat([tf.zeros(cat_filenames.shape,dtype=tf.int32),
                     tf.ones(dog_filenames.shape,dtype=tf.int32)],
                     axis=-1)
    return filenames,labels
def decode_image_and_resize(filename,label):
    image_string=tf.io.read_file(filename)
    image_decode=tf.image.decode_jpeg(image_string)
    image_resize=tf.image.resize(image_decode,[224,224])/255.0
    return image_resize,label

In [3]:
%matplotlib
import os
import matplotlib.pyplot as plt
data_dir='../kaggle/train/'
filenames,labels=read_image_filenames(data_dir)

Using matplotlib backend: Qt5Agg


In [4]:
dataset=tf.data.Dataset.from_tensor_slices((filenames,labels))
dataset=dataset.map(
    map_func=decode_image_and_resize,
    num_parallel_calls=tf.data.experimental.AUTOTUNE
)
sub_data=dataset.take(3)
for x,y in sub_data:
    print(x.numpy(),y.numpy())

[[[0.79457283 0.64163166 0.33967087]
  [0.80332637 0.6503852  0.34842438]
  [0.81600136 0.6630602  0.36109945]
  ...
  [0.95446384 0.7976011  0.4642677 ]
  [0.9445028  0.7954832  0.4778361 ]
  [0.93574923 0.78672963 0.4690826 ]]

 [[0.79457283 0.64163166 0.33967087]
  [0.80332637 0.6503852  0.34842438]
  [0.81600136 0.6630602  0.36109945]
  ...
  [0.9565476  0.8003151  0.46698177]
  [0.9445203  0.7955007  0.4778536 ]
  [0.93576676 0.78674716 0.4691001 ]]

 [[0.79457283 0.64163166 0.33967087]
  [0.80332637 0.6503852  0.34842438]
  [0.81600136 0.6630602  0.36109945]
  ...
  [0.9543289  0.80295867 0.47491246]
  [0.94842434 0.79940474 0.48175767]
  [0.9396708  0.7906512  0.47300416]]

 ...

 [[0.60264367 0.48107502 0.21832994]
  [0.60597    0.48440135 0.22165626]
  [0.6104868  0.48891816 0.22617307]
  ...
  [0.00784314 0.01176471 0.        ]
  [0.00784314 0.01176471 0.        ]
  [0.00784314 0.01176471 0.        ]]

 [[0.59609604 0.4745274  0.21178229]
  [0.6000149  0.4784463  0.21570121]


In [5]:
for x,y in sub_data:
    plt.title(y.numpy())
    plt.imshow(x.numpy())
    plt.show()

In [6]:
buffer_size=25000
dataset=dataset.shuffle(buffer_size)
dataset=dataset.batch(8)

In [7]:
#（卷积-卷积-池化）*5 + 全连接*3+ softmax   VGG模型
#我们将卷积池化过程权重全部保留，自己编写全连接与softmax层
def vgg_16(input_shape=(224,224,3)):
    vgg16=tf.keras.applications.VGG16(include_top=False,
                                           weights='imagenet',
                                           input_shape=input_shape)
    for layer in vgg16.layers:
        layer.trainable=False
    last=vgg16.output
    x=tf.keras.layers.Flatten()(last)
    x=tf.keras.layers.Dense(128,activation='relu')(x)
    x=tf.keras.layers.Dropout(0.3)(x)
    x=tf.keras.layers.Dense(32,activation='relu')(x)
    x=tf.keras.layers.Dropout(0.3)(x)
    x=tf.keras.layers.Dense(2,activation='softmax')(x)
    model=tf.keras.models.Model(inputs=vgg16.inputs,outputs=x)
    model.summary()
    return model

In [8]:
model=vgg_16()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
train_history=model.fit(dataset,epochs=5,verbose=1)

Epoch 1/5
